In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import matplotlib.ticker as plticker
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from collections import defaultdict
from typing import Optional
from scipy.integrate import solve_ivp
from scipy.fftpack import diff as psdiff
import torch
import pytorch_lightning as pl

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympde'))

from data.dataset import PDEDataset, PDEDataModule
from data.utils import d_to_LT

from model.setup import setup_model
from model.networks.fno import FNO1d
from model.learner import Learner

from viz.plot_pde_data import plot_1ds

from run import parse_options, main

from data.utils import d_to_coords
from data.pde_data_aug import augment_pde1, augment_KdV

Set seed!


In [15]:
args = parse_options(notebook=True)
args.pde_name = 'KdV'
args.seed = 2
args.version = f'{args.pde_name}_seed{args.seed}'
args.train = True
args.num_workers = 7
args.local = True
args.batch_size = 1
args.do_return = True
args.persistent_workers = True
args.max_epochs = 20
args.n_splits = [20, 20, 20]


In [18]:
args.generators = True
args.do_return = False
model, trainer, datamodule = main(args)


/home/eliasd/.conda/envs/sympde/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/eliasd/.conda/envs/sympde/lib/python3.10/site- ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-11-23 22:23:22.211785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 22:23:22.211890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 22:23:22.213824: E external/local_xla/xla/stream_executor/cuda/cuda_blas

Augmenting KdV!
Not augmenting KdV!
Not augmenting KdV!
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]Augmenting KdV!
Not augmenting KdV!
Not augmenting KdV!


/home/eliasd/.conda/envs/sympde/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
/home/eliasd/.conda/envs/sympde/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/eliasd/.conda/envs/sympde/lib/python3.10/site- ...


Testing: |          | 0/? [00:00<?, ?it/s]

In [14]:
# args.generators = True
# args.do_return = True

# # model, trainer, datamodule = main(args)
# datamodule = main(args)
# datamodule.setup('test')

In [8]:
args.version = 'aug1_seed2_epochs20'
args.generators = True

model, trainer, datamodule = main(args)

Seed set to 2


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type  | Params
-------------------------------
0 | net  | FNO1d | 10.9 M
-------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.406    Total estimated model params size (MB)


/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pyt

Epoch 0:   0%|          | 0/160 [00:00<?, ?it/s] 

/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch 19: 100%|██████████| 160/160 [00:22<00:00,  7.02it/s, v_num=hs20, train_loss=0.126, val_loss=0.0709] 

`Trainer.fit` stopped: `max_epochs=20` reached.


Testing DataLoader 0: 100%|██████████| 20/20 [00:00<00:00, 22.69it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.06806756556034088
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [9]:
args.version = 'aug0_seed2_epochs20'
args.generators = False

model, trainer, datamodule = main(args)

Seed set to 2
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type  | Params
-------------------------------
0 | net  | FNO1d | 10.9 M
-------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.406    Total estimated model params size (MB)


/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/elias/anaconda3/envs/sympde/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pyt

Epoch 19: 100%|██████████| 160/160 [00:22<00:00,  7.12it/s, v_num=hs20, train_loss=0.0612, val_loss=0.0393]

`Trainer.fit` stopped: `max_epochs=20` reached.


Testing DataLoader 0: 100%|██████████| 20/20 [00:00<00:00, 22.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.042173244059085846
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
assert False

AssertionError: 

In [ ]:
datamodule.setup(stage='test')
batch = next(iter(datamodule.train_dataloader()))

In [ ]:
# set to inference mode
model.eval()


In [ ]:
model = model.eval()
y_pred, y = model(batch)
y_pred = y_pred.detach().cpu().numpy()

In [ ]:
y_pred.shape

In [ ]:
model.log_fig(batch, y_pred.detach())

## G-FNO

In [ ]:
num_channels = 1
modes = 12
width = 10 # for G-FNO p4, for G-FNO p4m then 7
T_in = initial_step = 10 # input time (history)
reflection = False # False if p4, if p4m then True
grid_type = 'symmetric'
strategy = 'teacher_forcing'
T = 20
noise_std = 0

In [ ]:
from model.networks.gfno import GFNO2d, pde_data

In [ ]:
model = GFNO2d(
    num_channels=num_channels,
    modes=modes,
    width=width,
    initial_step=initial_step,
    reflection=reflection,
    grid_type=grid_type,
)

In [ ]:
# x = us[:, :10, :]
# model(x)

In [ ]:
data_path = '../ext_repos/G-FNO/data'

train_path = os.path.join(data_path, 'ns_V0.01_N10_T30_cos4_super.mat')
data = scipy.io.loadmat(os.path.expandvars(train_path))['u'].astype(np.float32)
data = torch.from_numpy(data).float()
print(data.shape)
data = data.unsqueeze(-1)
print(data.shape)

In [ ]:
train_dataset = pde_data(data, strategy=strategy, T_in=T_in, T_out=T, std=noise_std)

In [ ]:
x, y = train_dataset[0]

In [ ]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)

In [ ]:
xx, yy = next(iter(dataloader))

In [ ]:
x_pred = model(xx)

In [ ]:
x_pred.shape

In [ ]:
x.shape, y.shape